## load model

In [1]:

from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq


tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


c:\Users\User\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## load dataset

In [2]:
from datasets import load_dataset


ds = load_dataset("keivalya/MedQuad-MedicalQnADataset")
ds

DatasetDict({
    train: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 16407
    })
})

## split the dataset

In [3]:
ds=ds['train'].train_test_split(test_size=0.1)
ds


DatasetDict({
    train: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 14766
    })
    test: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 1641
    })
})

## processing dataset

In [4]:
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   inputs = [prefix + doc for doc in examples["Question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
  
   labels = tokenizer(text_target=examples["Answer"], 
                      max_length=512,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

tokenized_dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/14766 [00:00<?, ? examples/s]

Map: 100%|██████████| 1641/1641 [00:01<00:00, 1423.68 examples/s]


## compute_metrics

In [5]:
import nltk
import evaluate
import numpy as np

nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

## fine-tuning

In [6]:
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,  
    per_device_train_batch_size=2,  
    per_device_eval_batch_size=2, 
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_steps=20,
    save_total_limit=3,
    predict_with_generate=True,
    push_to_hub=False,
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()




c:\Users\User\miniconda3\envs\transformers\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 22/22149 [00:03<47:55,  7.69it/s]  

{'loss': 3.4502, 'grad_norm': 3.9185245037078857, 'learning_rate': 1.9981940493927492e-05, 'epoch': 0.0}


  0%|          | 41/22149 [00:06<54:15,  6.79it/s]  

{'loss': 3.3094, 'grad_norm': 2.9170382022857666, 'learning_rate': 1.9963880987854986e-05, 'epoch': 0.01}


  0%|          | 61/22149 [00:09<48:29,  7.59it/s]

...

100%|██████████| 22149/22149 [57:42<00:00,  8.01it/s]c:\Users\User\miniconda3\envs\transformers\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                     
100%|██████████| 22149/22149 [1:01:15<00:00,  8.01it/s]

{'eval_loss': 1.9784570932388306, 'eval_rouge1': 0.14780574818881195, 'eval_rouge2': 0.09150305729120847, 'eval_rougeL': 0.13552779011805605, 'eval_rougeLsum': 0.14225015139085656, 'eval_runtime': 212.8324, 'eval_samples_per_second': 7.71, 'eval_steps_per_second': 3.857, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
100%|██████████| 22149/22149 [1:01:16<00:00,  6.02it/s]

{'train_runtime': 3676.5869, 'train_samples_per_second': 12.049, 'train_steps_per_second': 6.024, 'train_loss': 2.333870041983419, 'epoch': 3.0}


TrainOutput(global_step=22149, training_loss=2.333870041983419, metrics={'train_runtime': 3676.5869, 'train_samples_per_second': 12.049, 'train_steps_per_second': 6.024, 'total_flos': 386499761479680.0, 'train_loss': 2.333870041983419, 'epoch': 3.0})

## inference

In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
last_checkpoint = "google/flan-t5-small"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint).to("cuda")
finetuned_tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)
question="what are marine toxins?"

input_text = "Please answer this medical related question: "+question
input_ids = finetuned_tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = finetuned_model.generate(
    input_ids,
    max_length=200,
    min_length=100,
    repetition_penalty=2.0
)
answer = finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)
from textwrap import fill

print(fill(answer, width=100))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


toxins from the marine environment.com/selective-chemical reactions.html, a chemical reaction that
is toxic to humans and animals.com/selective-chemical reactions.com/selective-chemical
reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical
reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical
reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical
reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical
reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-


In [2]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")
question="what are marine toxins?"

input_text = "Please answer this medical related question: "+question
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(
    input_ids,
    max_length=200,
    min_length=100,
    repetition_penalty=2.0
)
print(tokenizer.decode(outputs[0]))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<pad>toxins from the marine environment.com/selective-chemical reactions.html, a chemical reaction that is toxic to humans and animals.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-chemical reactions.com/selective-
